<img src="https://docs.xarray.dev/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">


# Weighted Reductions

Xarray supports [weighted reductions](https://docs.xarray.dev/en/stable/user-guide/computation.html#weighted-array-reductions).

For demonstration, we will create a “weights” array proportional to cosine of latitude.
Modulo a normalization, this is the correct area-weighting factor for data on a regular lat-lon grid.


In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

%config InlineBackend.figure_format='retina'

In [ ]:
ds = xr.tutorial.load_dataset("ersstv5")
ds

In [ ]:
weights = np.cos(np.deg2rad(ds.lat))
display(weights.dims)

weights.plot()

## Manual weighting

Thanks to the automatic broadcasting and alignment discussed earlier, if we multiply this by SST, it “just works,” and the arrays are broadcasted properly:

In [ ]:
(ds.sst * weights).dims

We could imagine computing the weighted spatial mean of SST manually.

In [ ]:
sst_mean = (ds.sst * weights).sum(dim=("lon", "lat")) / weights.sum(dim="lat")
sst_mean.plot()
plt.title("This is wrong!")

That would be wrong, however, because the denominator (`weights.sum(dim='lat')`)
needs to be expanded to include the `lon` dimension and modified to account for
the missing values (land points).


## The `weighted` method
In general, weighted reductions on multidimensional arrays are complicated. To
make it a bit easier, Xarray provides a mechanism for weighted reductions. 

It does this by creating a special intermediate `DataArrayWeighted` object, to
which different reduction operations can applied.



In [ ]:
sst_weighted = ds.sst.weighted(weights)
sst_weighted


<div class="alert alert-info">
    Xarrays' computation methods (groupby, groupby_bins, rolling, coarsen, weighted) all return special objects that represent the basic underlying computation pattern. For e.g. `sst_weighted` above is a `DatasetWeighted` object that represents the weighting by `weights` of the data in `ds.sst`. It is usually helpful to save and reuse these objects for multiple operations (e.g. a mean and standard deviation calculation).
</div>

In [ ]:
sst_weighted.mean(dim=("lon", "lat")).plot()
plt.title("Correct Global Mean SST");

A handful of reductions have been implemented: [mean, sum, std, var](https://docs.xarray.dev/en/stable/api.html#weighted-objects).